In [31]:
sc

In [32]:
# Set up directories to the datasets
REVIEW_PATH = 'yelp_review.csv'

In [33]:
def extractRow(index, lines):
    import csv
    if index == 0:
        lines.next()
        
    reader = csv.reader(lines)
    for row in reader:
        yield row

In [34]:
reviews_rdd = sc.textFile("yelp_review.csv", use_unicode=False)
reviews_rdd_new = reviews_rdd.mapPartitionsWithIndex(extractRow).cache()

In [35]:
# Create the schema necessary for the creation of a DataFrame
from pyspark.sql.types import *
schema = StructType([
    StructField("review_id", StringType(), False), 
    StructField("user_id", StringType(), False),
    StructField("business_id", StringType(), False),    
    StructField("stars", StringType(), True),    
    StructField("date", StringType(), True),    
    StructField("text", StringType(), True),    
    StructField("useful", StringType(), True),    
    StructField("funny", StringType(), True),    
    StructField("cool", StringType(), True)])

In [36]:
# Create a dataframe using the RDD and the previously declared schema
#reviews_df = spark.createDataFrame(reviews_rdd_new, schema)
reviews_df = sqlContext.createDataFrame(reviews_rdd_new, schema)

reviews_df.printSchema()

root
 |-- review_id: string (nullable = false)
 |-- user_id: string (nullable = false)
 |-- business_id: string (nullable = false)
 |-- stars: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)



In [37]:
# Cast all columns to the appropriate datatype
reviews_df = reviews_df.select(reviews_df['review_id'], 
                             reviews_df['user_id'],
                             reviews_df['business_id'], 
                             reviews_df['stars'].cast(IntegerType()), 
                             reviews_df['date'].cast(DateType()), 
                             reviews_df['text'], 
                             reviews_df['useful'].cast(IntegerType()),
                             reviews_df['funny'].cast(IntegerType()),
                             reviews_df['cool'].cast(IntegerType()),
                            ).cache()

reviews_df.take(1)

[Row(review_id=u'vkVSCC7xljjrAI4UGfnKEQ', user_id=u'bv2nCi5Qv5vroFiqKGopiw', business_id=u'AEx2SYEUJmTxVVB18LlCwA', stars=5, date=datetime.date(2016, 5, 28), text=u"Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. Staff was very helpful and friendly.", useful=0, funny=0, cool=0)]

In [40]:
from functools import reduce
new_df = reviews_df.filter(~reduce(lambda x, y: x & y, [reviews_df[c].isNull() for c in reviews_df.columns]))
new_df.show(2)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
only showing top 2 rows



In [41]:

print(new_df.count())

#total = new_df.select("review_id").agg({"review_id": "sum"}).collect().pop()['sum(review_id)']
#print(total)
#new_df = new_df.groupby('business_id').count()
#new_df.show(2)

Py4JJavaError: An error occurred while calling o827.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 24.0 failed 1 times, most recent failure: Lost task 3.0 in stage 24.0 (TID 59, localhost, executor driver): java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(Unknown Source)
	at java.net.SocketInputStream.read(Unknown Source)
	at java.io.BufferedInputStream.read1(Unknown Source)
	at java.io.BufferedInputStream.read(Unknown Source)
	at java.io.DataInputStream.readFully(Unknown Source)
	at java.io.DataInputStream.readFully(Unknown Source)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:169)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:152)
	at org.apache.spark.InterruptibleIterator.next(InterruptibleIterator.scala:43)
	at scala.collection.Iterator$$anon$12.nextCur(I

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 61396)
----------------------------------------


Traceback (most recent call last):
  File "C:\Users\migue\Anaconda2\lib\SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\migue\Anaconda2\lib\SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\migue\Anaconda2\lib\SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\migue\Anaconda2\lib\SocketServer.py", line 652, in __init__
    self.handle()
  File "C:\spark\python\pyspark\accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "C:\spark\python\pyspark\serializers.py", line 575, in read_int
    length = stream.read(4)
  File "C:\Users\migue\Anaconda2\lib\socket.py", line 384, in read
    data = self._sock.recv(left)
error: [Errno 10054] An existing connection was forcibly closed by the remote host


In [11]:
#val = reviews_df.na.drop(subset=['business_id'])
#val = reviews_df.groupBy('business_id').count()

In [12]:
#val.show()

Py4JJavaError: An error occurred while calling o187.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 1 times, most recent failure: Lost task 3.0 in stage 4.0 (TID 11, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\python\pyspark\sql\session.py", line 509, in prepare
    verify_func(obj, schema)
  File "C:\spark\python\lib\pyspark.zip\pyspark\sql\types.py", line 1358, in _verify_type
    "length of fields (%d)" % (len(obj), len(dataType.fields)))
ValueError: Length of object (6) does not match with length of fields (9)

	at org.apache.spark.api.python.Python